In [ ]:
def setKeywords(self,method='adjAdv',wordCount=10,startCount=0):
        '''
        function to automatically assign keywords if manual ones have not been assigned
        
        Inputs
        ======
        method: string
            Method used to pick automatically defined keywords. Choose from:
            adjAdv- picks most common adj and adv in text (default and 
                catch if other method doesn't exist)
            judgement-Under development
            SME - get a list of words from subject matter expert
        wordCount: int
            Number of keywords returned (default 10)
        startCount: int
            Index where keywords are extracted (default 0 i.e. start of list)
        
        Attributes
        ==========
        keywords: list
            List of keywords automatically generated (can also be assigned 
            outside of function manually)
        '''
        #Save input values
        self.keywordCount=wordCount
        self.keywordStar=startCount
        
        #Judgement method
        if method=='judgement':
            posList=nounList+tagFilterList
            #Define target dict
            targetDict={}
            for fileName in self.fileList:
                for judgementStr in self.judgements[fileName]:
                    tagList=tagger.tag(nltk.word_tokenize(judgementStr))
            
                    
                    #Loop through each tag in list and get count of tag and word
                    for tag in tagList:
                        if tag[1] in posList:
                            word=str.lower(''.join([c for c in tag[0] if c not in string.punctuation]))
                            #Stem words if useStem True
                            newStopWords=stopWords
                            if self.useStem:
                                word=stemmer.stem(word)
                                newStopWords=[stemmer.stem(x) for x in stopWords]
                             
                            #Remove stopwords if useStopwords ==False
                            if not self.useStopwords: 
                                newStopWords.append("")

                                
                            #Filter out codecerrors
                            if word not in ['codecerror']+[' ']+newStopWords:
                                try:
                                    targetDict[word]=targetDict[word]+1
                                except:
                                    targetDict[word]=1
            #Create data frame with counts and sort
            targetDF=pd.DataFrame([[k,v] for k,v in targetDict.items()],columns=['word','count'])
            targetDF.sort(['count'],inplace=True,ascending=False)
            
            #Create keywords based on startCount and wordCount
            self.keywords=list(targetDF['word'])[startCount:wordCount+startCount]
            
        #Default to 'adjAdv'
        elif method=='adjAdv':
            #Get total text string
            txtString=''.join([x for x in self.rawText.values()])
            
            #Get total tag list
            tagList=tagger.tag(nltk.word_tokenize(txtString))
            
            #Define target dict
            targetDict={}
            
            #Loop through each tag in list and get count of tag and word
            for tag in tagList:
                if tag[1] in tagFilterList:
                    word=str.lower(''.join([c for c in tag[0] if c not in string.punctuation]))
                    #Filter out codecerrors
                    if word != 'codecerror':
                        try:
                            targetDict[word]=targetDict[word]+1
                        except:
                            targetDict[word]=1
        
            #Create data frame with counts and sort
            targetDF=pd.DataFrame([[k,v] for k,v in targetDict.items()],columns=['word','count'])
            targetDF.sort(['count'],inplace=True,ascending=False)
            
            #Create keywords based on startCount and wordCount
            self.keywords=list(targetDF['word'])[startCount:wordCount+startCount]
        elif method == 'SME':
            
        else:
            print('ERROR: Method not found')

In [25]:
# Keyword Implementation Prototype

import pandas as pd

# Pull data from the csv file
filepath = "/Users/samanthagarofalo/Documents/Data Science/Capstone/Keywords.csv"

# Create dataframe with manually entered keywords
targetDF = pd.read_csv(filepath)

# User input to select the group that we are looking at keywords for
group = input("Which group would you like to look at? ")

keywords = list(targetDF.Keywords[targetDF['Group'] == group])

for element in keywords:
    keywords = element.split(' ')
    
print(keywords)

Which group would you like to look at? DSI
['big', 'data', 'capstone', 'advisor', 'graduate', 'dell', 'science', 'career']
